In [ ]:
import torch
import numpy as np

# check CUDA availability
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print("CUDA not available, training on CPU")
else:
    print("CUDA available, training on GPU")